# 검색

검색 알고리즘은 순차 검색과 이진 검색이 있다. 순차 검색은 배열이 정렬되어 있지 않거나, 연결 리스트와 같이 입력이 동적으로 할당되는 경우에 사용된다. 이진 검색은 배열이 정렬되어 있는 경우 최선의 선택이다. 해시 테이블은 보조 메모리 공간을 사용하지만, 키를 이용하면 O(1)에 원하는 값을 검색할 수 있다. 

## 10.1 정렬되지 않은 배열

### 10.1.1 순차 검색
순차 검색은 최선의 경우 시간복잡도는 O(1)이고, 평균은 O(n/2) 최악은 o(n)이다. 리스트 안에 검색하려는 항목이 없으면 최악/최선/평균 모두 O(n)이다. 


In [3]:
# 정렬되지 않은 리스트 검색
def sequential_search(seq,n):
    for item in seq:
        if item == n:
            return True
    return False

def test_sequential_search():
    seq = [1,5,6,8,3]
    n1 = 5
    n2 = 7
    assert(sequential_search(seq,n1) is True)
    assert(sequential_search(seq,n2) is False)
    print('테스트 통과')

if __name__ == '__main__':
    test_sequential_search()
    

테스트 통과


리스트가 정렬되어 있다면, 리스트 안에 검색하려는 항목이 없는 경우에도 검색하려는 항목이 있을 때와 같은 실행 시간을 가질 수 있다. 

In [5]:
# 정렬된 리스트 검색
def ordered_sequential_search(seq, n):
    item = 0
    for item in seq:
        if item > n:
            return False
        if item == n:
            return True
    return False

def test_ordered_sequential_search():
    seq = [1,2,4,5,6,8,10]
    n1 = 10
    n2 = 7
    assert(ordered_sequential_search(seq,n1) is True)
    assert(ordered_sequential_search(seq,n2) is False)
    print('테스트 통과')

if __name__ == '__main__':
    test_ordered_sequential_search()

테스트 통과


### 10.1.2 빠른 선택과 순서통계량
퀵 정렬 알고리즘을 약간 수정해서 리스트에서 k번째로 작은 항목을 찾는다. 이러한 숫자 k를 k번째 순서통계량(최솟값, 최댓값, 중앙값 등)이라 부른다. 최악의 경우 O(n)의 시간복잡도를 가진다. 

In [11]:
import random

def quick_select_cache(seq, k):
    len_seq = len(seq)
    if len_seq < 2:
        return seq[0]

# 피벗을 무작위로 선택할 수 있다.
# pivot = random.choice(seq)
    ipivot = len_seq // 2
    pivot = seq[ipivot]

    smallerList = [x for i, x in enumerate(seq) if x <= pivot and i != ipivot]
    largerList = [x for i, x in enumerate(seq) if x > pivot and i != ipivot]

    m = len(smallerList)
    if k == m:
        return pivot
    elif k < m:
        return quick_select_cache(smallerList, k)
    else: 
        return quick_select_cache(largerList, k-m-1)
    
def swap(seq,x,y):
    seq[x], seq[y] = seq[y], seq[x]
    
def quick_select(seq, k, left = None, right = None):
    left = left or 0
    right = right or len(seq) - 1
    # ipivot = random.randint(left,right)
    ipivot = len(seq) // 2
    pivot = seq[ipivot]
    
    # 피벗을 정렬 범위 밖으로 이동한다. 
    swap(seq, ipivot,right)
    swapIndex, i = left, left
    
    while i < right:
        if pivot < seq[i]:
            swap(seq, i, swapIndex)
            swapIndex += 1
        i += 1
        
    # 피벗 위치를 확정한다.    
    swap(seq, right, swapIndex)
    
    # 피벗 위치 확인
    rank = len(seq) - swapIndex
    if k == rank:
        return seq[swapIndex]
    elif k < rank:
        return quick_select(seq, k, swapIndex+1, right)
    else:
        return quick_select(seq,k,left,swapIndex-1)

if __name__ == '__main__':
    seq = [3,7,2,1,4,6,5,10,9,11]
    k = len(seq) // 2
    print(sorted(seq))
    print(quick_select_cache(seq, k - 1))
    print(quick_select(seq,k))
    # 중앙값 출력을 위해 넘파이 사용
    import numpy as np
    print(np.median(seq))

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11]
5
5
5.5


일반적으로 배열의 중앙에 있는 값보다 '큰'값으로 중앙값을 정의할 수 있다. 이러한 정의는 가장 가까운 이웃 또는 최단 경로를 찾는 등의 문제를 풀 때 중요하다. 

## 10.2 정렬된 배열
### 10.2.1 이진 검색

이진 검색은 정렬된 배열 내에서 지정된 입력값의 위치(키)를 찾는다. 이진 검색은 알고리즘의 각 단계에서 입력값과 배열 중간 요소를 비교한다. 입력값과 중간 요소가 일치하면 배열의 위치가 반환된다. 입력값이 중간 요소보다 작으면, 중간 요소의 왼쪽 하위 배열
에서 검색 과정을 반복한다. 반대의 경우엔 중간 요소의 오른쪽 하위 배열에서 검색 과정을 반복한다. 시간복잡도는  O(log n)이다.

In [13]:
# 재귀 함수
def binary_search_rec(seq, target, low, high):
    if low > high:
        return None
    mid = (low + high) // 2
    if target == seq[mid]:
        return mid
    elif target < seq[mid]:
        return binary_search_rec(seq,target,low,mid - 1)
    else: 
        return binary_search_rec(seq,target,mid+1,high)
    
# 반복문
def binary_search_iter(seq, target):
    high, low = len(seq), 0
    while low < high:
        mid = (high + low) // 2
        if target == seq[mid]:
            return mid
        elif target < seq[mid]:
            high = mid
        else:
            low = mid + 1
    return None

def test_binary_search():
    seq = [1,2,5,6,7,10,12,12,14,15]
    target = 6
    assert(binary_search_iter(seq, target) == 3)
    assert(binary_search_rec(seq,target,0,len(seq)) == 3)
    print('테스트 통과')
    
if __name__ == '__main__':
    test_binary_search()

테스트 통과


### 10.2.2 bisect 모듈

예제에서 bisect() 함수는 임력값 5의 리스트 위치(인덱스+1)을 반환한다.그 밖에도 bisect_right()와 bisect_left() 함수가 있다. 

In [15]:
from bisect import bisect
l = [0,3,4,5]
print(bisect(l,5))
# 빈 리스트 혹은 값이 없는 예외의 경우
print(bisect(l,-1))
print(bisect(l,1))
print(bisect(l,7))
print(bisect([],1))

4
0
1
4
0


## 10.3 연습문제

### 10.3.1 행렬 검색

In [16]:
def find_elem_matrix_bool(m1, value):
    found = False
    row = 0
    col = len(m1[0]) - 1
    while row < len(m1) and col >= 0:
        if m1[row][col] == value:
            found = True
            break
        elif m1[row][col] > value:
            col -= 1
        else:
            row += 1
    return found

def test_find_elem_matrix_bool():
    m1 = [[1,2,8,9],[2,4,9,12],[4,7,10,13],[6,8,11,15]]
    assert(find_elem_matrix_bool(m1,8) is True)
    assert(find_elem_matrix_bool(m1,3) is False)
    m2 = [[0]]
    assert(find_elem_matrix_bool(m2,0) is True)
    print('테스트 통과')

if __name__ == '__main__':
    test_find_elem_matrix_bool()

테스트 통과


In [27]:
def searching_in_a_matrix(m1, value):
    rows = len(m1)
    cols = len(m1[0])
    lo = 0
    hi = rows*cols
    while lo < hi:
        mid = (lo + hi) // 2
        row = mid // cols
        col = mid % cols
        v = m1[row][col]
        if v == value:
            return True
        elif v > value:
            hi = mid
        else:
            lo = mid + 1
    return False

def test_seraching_in_a_matrix():
    a = [[1,3,5],[7,9,11],[13,15,17]]
    import numpy as np
    b = np.array([(1,2),(3,4)])
    assert(searching_in_a_matrix(a,13) is True)
    assert(searching_in_a_matrix(a,14) is False)
    assert(searching_in_a_matrix(b,3) is True)
    assert(searching_in_a_matrix(b,5) is False)
    print('테스트 통과')

if __name__ == '__main__':
    test_seraching_in_a_matrix()

테스트 통과


### 10.3.2 단봉형 배열

배열 요소들의 산포도를 그렸을 때 값이 증가했다가 다시 감소하는 곡선인 경우 이 배열을 단봉형이라고 한다. 

In [31]:
def find_max_unimodal_array(A):
    if len(A) <= 2:
        return None
    left = 0
    right = len(A) - 1
    while right > left + 1 :
        mid = (left + right) // 2
        if A[mid] > A[mid - 1] and A[mid] > A[mid+1]:
            return A[mid]
        elif A[mid] > A[mid - 1] and A[mid] < A[mid + 1]:
            left = mid
        else:
            right = mid
    return None

def test_find_max_unimodal_array():
    seq = [1,2,5,6,7,10,12,9,8,7,6]
    assert(find_max_unimodal_array(seq) == max(seq))
    print('테스트 통과')
    
if __name__ == '__main__':
    test_find_max_unimodal_array()

테스트 통과


### 10.3.3 제곱근 계산하기

In [34]:
def find_sqrt_bin_search(n, error = 0.001):
    lower = n < 1 and n or 1
    upper = n < 1 and 1 or n
    mid = lower + (upper - lower) / 2.0
    square = mid * mid
    while abs(square - n) > error:
        if square < n:
            lower = mid
        else:
            upper = mid
        mid = lower + (upper - lower) / 2.0
        square = mid * mid
    return mid

if __name__ == '__main__':
    a = 2
    b = 9
    import math
    print(math.sqrt(a))
    print(find_sqrt_bin_search(a))
    print(math.sqrt(b))
    print(find_sqrt_bin_search(b))

1.4142135623730951
1.4140625
3.0
3.0


### 10.3.4 빈도 계산하기

In [37]:
def find_time_occurrence_list(seq,k):
    index_some_k = binary_search_iter(seq,k)
    count = 1
    sizet = len(seq)
    for i in range(index_some_k + 1, sizet):
        if seq[i] == k:
            count += 1
        else:
            break
    for i in range(index_some_k-1,-1,-1):
        if seq[i] == k:
            count += 1
        else:
            break
    return count

def test_find_time_occurrence_list():
    seq = [1,2,2,2,2,2,2,5,6,6,7,8,9]
    k = 2
    assert(find_time_occurrence_list(seq,k) == 6)
    print('테스트 통과')

if __name__ == '__main__':
    test_find_time_occurrence_list()

테스트 통과


### 10.3.5 교집합 구하기

In [44]:
# python set 사용
def intersection_two_arrays_sets(seq1,seq2):
    set1 = set(seq1)
    set2 = set(seq2)
    return set1.intersection(set2)

# 병합 정렬 사용
def intersection_two_arrays_ms(seq1,seq2):
    res = []
    while seq1 and seq2:
        if seq1[-1] == seq2[-1]:
            res.append(seq1.pop())
            seq2.pop()
        elif seq1[-1] > seq2[-1]:
            seq1.pop()
        else:
            seq2.pop()
    res.reverse()
    return res

# 이진 검색 사용
def intersection_two_arrays_bs(seq1, seq2):
    if len(seq1) > len(seq2):
        seq, key = seq1, seq2
    else:
        seq, key = seq2, seq1
    intersec = []
    for item in key:
        if binary_search_iter(seq,item):
            intersec.append(item)
    return intersec

def test_intersection_two_arrays():
    seq1 = [1,2,3,5,7,8]
    seq2 = [3,5,6]
    assert(set(intersection_two_arrays_sets(seq1,seq2)) == set([3,5]))
    assert(intersection_two_arrays_bs(seq1,seq2) == [3,5])
    assert(intersection_two_arrays_ms(seq1,seq2) == [3,5])
    print('테스트 통과')

if __name__ == '__main__':
    test_intersection_two_arrays()

테스트 통과
